In [ ]:
# ===========================================
# Earth Engine Initialization and Authentication
# ===========================================

import ee

# Method 1: Interactive authentication (for local development or Jupyter Notebooks)
# This method will open a browser window to log in with your Google account.
# Use this when running locally for the first time or if no credentials are stored.
ee.Authenticate(quiet=True)

# Initialize Earth Engine (automatically uses default credentials if already authenticated)
ee.Initialize()

# Test the initialization
print(ee.String('GEE initialized').getInfo())

In [ ]:
# Method 2: Service account authentication (for automated pipelines or server deployment)
# Note: The service account JSON key file must be located in the same directory as this script.

credentials = ee.ServiceAccountCredentials(
     'changsome.1@gmail.com',  # Service account email
     'global-flood-mapping-db762f1c1e40.json'  # Path to service account key file
 )
ee.Initialize(credentials)

# Test the initialization
print(ee.String('GEE initialized').getInfo())

In [ ]:
# ===========================================
# Interactive Map Initialization
# ===========================================

# Import geemap, interactive mapping
import geemap

# Initialize an interactive map using geemap
m = geemap.Map()

# Note: If this is the last line of a cell, you can omit 'display()' and just write 'm'
display(m)

In [ ]:
# ===========================================
# SAR Image Collection and Multi-Temporal Mean Visualization
# ===========================================

# Define a function to mask out the low-value edges of the SAR image
def mask_edge(image):
    """
    Masks the edge areas of a Sentinel-1 SAR image where backscatter is very low.
    
    Args:
        image (ee.Image): Input SAR image (VV polarization)
    
    Returns:
        ee.Image: Image with low-value edges masked out
    """
    edge = image.lt(-30.0)  # Identify areas with backscatter < -30 dB (likely noisy edges)
    masked_image = image.mask().And(edge.Not())  # Keep valid areas only
    return image.updateMask(masked_image)


# Create an ImageCollection of Sentinel-1 SAR images with VV polarization and IW mode
img_vv = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))  # VV polarization only
    .filter(ee.Filter.eq('instrumentMode', 'IW'))  # Interferometric Wide (IW) mode
    .select('VV')  # Select VV band
    .map(mask_edge)  # Apply edge masking function
)

# Split the collection into descending and ascending orbit tracks
desc = img_vv.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
asc = img_vv.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))

# Define seasonal date filters
spring = ee.Filter.date('2015-03-01', '2015-04-20')
late_spring = ee.Filter.date('2015-04-21', '2015-06-10')
summer = ee.Filter.date('2015-06-11', '2015-08-31')

# Create a composite image stacking the mean of each period (descending)
desc_change = ee.Image.cat(
    desc.filter(spring).mean(),
    desc.filter(late_spring).mean(),
    desc.filter(summer).mean(),
)

# Create a composite image stacking the mean of each period (ascending)
asc_change = ee.Image.cat(
    asc.filter(spring).mean(),
    asc.filter(late_spring).mean(),
    asc.filter(summer).mean(),
)

# Initialize the map
m = geemap.Map()
m.set_center(5.2013, 47.3277, 12)  # Center on a location (example: Dijon, France)

# Add the multi-temporal mean images to the map
m.add_layer(
    asc_change,
    {'min': -25, 'max': 5},
    'Multi-T Mean ASC',
    True
)
m.add_layer(
    desc_change,
    {'min': -25, 'max': 5},
    'Multi-T Mean DESC',
    True
)

# Display the map
m

## Flood Detection Using Sentinel-1 SAR Imagery

In [ ]:
# ===========================================
# Flood Detection Using Sentinel-1 SAR Imagery
# ===========================================

# Define the region of interest (ROI)
geometry = ee.Geometry.Polygon([
    [
        [106.34954329522984, -6.449380562588049],
        [107.33007308038609, -6.449380562588049],
        [107.33007308038609, -5.900522745264385],
        [106.34954329522984, -5.900522745264385]
    ]
])

# Initialize and center the map on the ROI
Map = geemap.Map()
Map.centerObject(geometry, 10)

# Sentinel-1 SAR image collection: before the flood event
sar_before = (
    ee.ImageCollection("COPERNICUS/S1_GRD")
    .filterDate('2019-12-20', '2019-12-29')
    .filterBounds(geometry)
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
    .select('VV')
    .map(lambda img: img.focalMean(60, 'square', 'meters')
         .copyProperties(img, img.propertyNames()))
)

# Sentinel-1 SAR image collection: after the flood event
sar_after = (
    ee.ImageCollection("COPERNICUS/S1_GRD")
    .filterDate('2019-12-30', '2020-01-03')
    .filterBounds(geometry)
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
    .select('VV')
    .map(lambda img: img.focalMean(60, 'square', 'meters')
         .copyProperties(img, img.propertyNames()))
)

# Compute the difference between before and after images
change = sar_before.min().subtract(sar_after.min())

# Load a permanent water mask from Dynamic World (2018–2021)
water_mask = (
    ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")
    .select('label')
    .filterDate('2018', '2021')
    .filterBounds(geometry)
    .mode()
    .eq(0)  # Class 0 = Water
    .Not()  # Invert: True where NOT permanent water
)

# Threshold change detection (>5 dB difference) and apply the water mask
thr = change.gt(5).updateMask(water_mask)
flooded = thr.updateMask(thr)

# Calculate flooded area in square kilometers
area_img = flooded.multiply(ee.Image.pixelArea().divide(1e6))
flood_area = area_img.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=geometry,
    scale=60
)

# Print the flooded area result
print(flood_area.getInfo())

# Add layers to the map
Map.addLayer(sar_before.min().clip(geometry), {}, "SAR Before")
Map.addLayer(sar_after.min().clip(geometry), {}, "SAR After")
Map.addLayer(change.clip(geometry), {}, "Change Detection")
Map.addLayer(flooded.clip(geometry), {"palette": ["blue"]}, "Detected Flooded Areas")

# Display the interactive map
Map

## Search for SAR overlay in an area

### Oriniginal method

In [ ]:
# Create a rectangular bounding box geometry
area = ee.Geometry.Rectangle([106.3, -6.4, 107.3, -5.9])

# Define the date range
start_date = '2019-12-30'
end_date = '2020-01-30'

# Load Sentinel-1 Image Collection for the given bounding box
sentinel1 = (ee.ImageCollection('COPERNICUS/S1_GRD')
             .filterBounds(area)
             .filterDate(start_date, end_date)
             .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
             .filter(ee.Filter.eq('instrumentMode', 'IW'))
             .select('VV'))

In [ ]:
image_list = sentinel1.toList(sentinel1.size())

for i in range(sentinel1.size().getInfo()): # 每次用 getInfo() 取 metadata 都要和 GEE server 請求一次
    image = ee.Image(image_list.get(i))
    segment_start_time = image.get('segmentStartTime')
    formatted_time = ee.Date(segment_start_time).format('YYYY-MM-dd HH:mm:ss').getInfo() if segment_start_time else "N/A" # 每次都要請求都要轉換一次時間
    print(f"Image {i+1} - segmentStartTime: {formatted_time}")

### Optimized version

In [ ]:
from datetime import datetime

# Define the bounding box and date range
area = ee.Geometry.Rectangle([106.3, -6.4, 107.3, -5.9])
start_date = '2019-12-30'
end_date = '2020-01-30'

# Load Sentinel-1 Image Collection
sentinel1 = (ee.ImageCollection('COPERNICUS/S1_GRD')
             .filterBounds(area)
             .filterDate(start_date, end_date)
             .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
             .filter(ee.Filter.eq('instrumentMode', 'IW'))
             .select('VV'))

# Get all segmentStartTime and IDs in one go .aggregate_array()
times = sentinel1.aggregate_array('segmentStartTime').getInfo()
ids = sentinel1.aggregate_array('system:id').getInfo()

# Print results
for i, (img_id, timestamp) in enumerate(zip(ids, times)):
    if timestamp:
        # Always treat as milliseconds since epoch
        dt = datetime.utcfromtimestamp(timestamp / 1000.0)
        formatted_time = dt.strftime('%Y-%m-%d %H:%M:%S')
    else:
        formatted_time = "N/A"
    print(f"Image {i+1} - ID: {img_id} - segmentStartTime: {formatted_time}")